In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functools as ft # 

pd.set_option('display.max_columns', None)

In [2]:
# read in 4 csv files from '../data/clean_data/' folder

# t1 == Medication Doses
t1 = pd.read_csv('/Users/danherman/Desktop/oud_treatment_outcome/data/clean_data/m_clean.csv')

# t2 == Self Reported Drug Use
t2 = pd.read_csv('/Users/danherman/Desktop/oud_treatment_outcome/data/clean_data/sru_agg_binned.csv')

# t3 == Urine Drug Screen
t3 = pd.read_csv('/Users/danherman/Desktop/oud_treatment_outcome/data/clean_data/u_clean.csv')

# target == Treatment Outcome
target = pd.read_csv('/Users/danherman/Desktop/oud_treatment_outcome/data/clean_data/target.csv')

In [3]:
t1.columns

Index(['patdeid', 'medication', 'total_visits', 'total_dose',
       'avg_weekly_dose', 'avg_daily_dose'],
      dtype='object')

In [4]:
t1 = t1.drop(columns=['total_visits', 'total_dose','avg_daily_dose'], axis=1)

In [5]:
t1[:5]

,patdeid,medication,avg_weekly_dose
0,1,2.0,208.0
1,2,2.0,78.0
2,3,1.0,523.0
3,4,2.0,213.0
4,6,2.0,16.0


In [6]:
# merge dfs, use .reduce() to treat all 4 dfs as one iterable

dfs = [t1, t2, t3, target]

t4 = ft.reduce(lambda left, right: pd.merge(left, right, on='patdeid'), dfs)

# drop eroneous columns
t4.drop(columns=['Unnamed: 0','VISIT'], axis=1, inplace=True)

In [7]:
display(t4.shape)
display(t4.head())

(1307, 604)

,patdeid,medication,avg_weekly_dose,sru_alcohol_0_above_mean,sru_alcohol_0_below_mean,sru_alcohol_0_mean,sru_alcohol_0_none_reported,sru_alcohol_12_above_mean,sru_alcohol_12_below_mean,sru_alcohol_12_mean,sru_alcohol_12_none_reported,sru_alcohol_16_above_mean,sru_alcohol_16_below_mean,sru_alcohol_16_mean,sru_alcohol_16_none_reported,sru_alcohol_20_above_mean,sru_alcohol_20_below_mean,sru_alcohol_20_none_reported,sru_alcohol_24_above_mean,sru_alcohol_24_below_mean,sru_alcohol_24_mean,sru_alcohol_24_none_reported,sru_alcohol_4_above_mean,sru_alcohol_4_below_mean,sru_alcohol_4_mean,sru_alcohol_4_none_reported,sru_alcohol_8_above_mean,sru_alcohol_8_below_mean,sru_alcohol_8_mean,sru_alcohol_8_none_reported,sru_cannabis_0_above_mean,sru_cannabis_0_below_mean,sru_cannabis_0_mean,sru_cannabis_0_none_reported,sru_cannabis_12_above_mean,sru_cannabis_12_below_mean,sru_cannabis_12_mean,sru_cannabis_12_none_reported,sru_cannabis_16_above_mean,sru_cannabis_16_below_mean,sru_cannabis_16_mean,sru_cannabis_16_none_reported,sru_cannabis_20_above_mean,sru_cannabis_20_below_mean,sru_cannabis_20_mean,sru_cannabis_20_none_reported,sru_cannabis_24_above_mean,sru_cannabis_24_below_mean,sru_cannabis_24_mean,sru_cannabis_24_none_reported,sru_cannabis_4_above_mean,sru_cannabis_4_below_mean,sru_cannabis_4_mean,sru_cannabis_4_none_reported,sru_cannabis_8_above_mean,sru_cannabis_8_below_mean,sru_cannabis_8_mean,sru_cannabis_8_none_reported,sru_cocaine_0_above_mean,sru_cocaine_0_below_mean,sru_cocaine_0_mean,sru_cocaine_0_none_reported,sru_cocaine_12_above_mean,sru_cocaine_12_below_mean,sru_cocaine_12_mean,sru_cocaine_12_none_reported,sru_cocaine_16_above_mean,sru_cocaine_16_below_mean,sru_cocaine_16_mean,sru_cocaine_16_none_reported,sru_cocaine_20_above_mean,sru_cocaine_20_below_mean,sru_cocaine_20_mean,sru_cocaine_20_none_reported,sru_cocaine_24_above_mean,sru_cocaine_24_below_mean,sru_cocaine_24_mean,sru_cocaine_24_none_reported,sru_cocaine_4_above_mean,sru_cocaine_4_below_mean,sru_cocaine_4_mean,sru_cocaine_4_none_reported,sru_cocaine_8_above_mean,sru_cocaine_8_below_mean,sru_cocaine_8_mean,sru_cocaine_8_none_reported,sru_amphetamine_0_above_mean,sru_amphetamine_0_below_mean,sru_amphetamine_0_none_reported,sru_amphetamine_12_above_mean,sru_amphetamine_12_below_mean,sru_amphetamine_12_none_reported,sru_amphetamine_16_above_mean,sru_amphetamine_16_below_mean,sru_amphetamine_16_none_reported,sru_amphetamine_20_below_mean,sru_amphetamine_20_none_reported,sru_amphetamine_24_above_mean,sru_amphetamine_24_below_mean,sru_amphetamine_24_none_reported,sru_amphetamine_4_above_mean,sru_amphetamine_4_below_mean,sru_amphetamine_4_mean,sru_amphetamine_4_none_reported,sru_amphetamine_8_below_mean,sru_amphetamine_8_none_reported,sru_methamphetamine_0_above_mean,sru_methamphetamine_0_below_mean,sru_methamphetamine_0_none_reported,sru_methamphetamine_12_above_mean,sru_methamphetamine_12_below_mean,sru_methamphetamine_12_mean,sru_methamphetamine_12_none_reported,sru_methamphetamine_16_above_mean,sru_methamphetamine_16_below_mean,sru_methamphetamine_16_mean,sru_methamphetamine_16_none_reported,sru_methamphetamine_20_above_mean,sru_methamphetamine_20_below_mean,sru_methamphetamine_20_none_reported,sru_methamphetamine_24_above_mean,sru_methamphetamine_24_below_mean,sru_methamphetamine_24_none_reported,sru_methamphetamine_4_above_mean,sru_methamphetamine_4_below_mean,sru_methamphetamine_4_mean,sru_methamphetamine_4_none_reported,sru_methamphetamine_8_above_mean,sru_methamphetamine_8_below_mean,sru_methamphetamine_8_none_reported,sru_opiates_0_above_mean,sru_opiates_0_below_mean,sru_opiates_0_mean,sru_opiates_0_none_reported,sru_opiates_12_above_mean,sru_opiates_12_below_mean,sru_opiates_12_mean,sru_opiates_12_none_reported,sru_opiates_16_above_mean,sru_opiates_16_below_mean,sru_opiates_16_none_reported,sru_opiates_20_above_mean,sru_opiates_20_below_mean,sru_opiates_20_none_reported,sru_opiates_24_above_mean,sru_opiates_24_below_mean,sru_opiates_24_mean,sru_opiates_24_none_repo

# Feature engineering
We will only keep the first 30 days of clinical data, to see if we can accurately predict if a patient will successfully complete treatment

In [8]:
# create list of columns to drop that we will remove from clinical data
t4_columns = [col for col in t4.columns if 'sru_' in col and ('_0_' not in col and '_4_' not in col)]


In [9]:
t4 = t4.drop(t4_columns, axis=1)

In [10]:
# create list of columns to drop that we will remove from clinical data
t4_columns2 = [col for col in t4.columns if 't_' in col and ('_0_'not in col and '_1_' not in col and '_2_' not in col and '_3_' not in col and '_4_' not in col)]

In [11]:
t4 = t4.drop(t4_columns2, axis=1)

In [12]:
t4.shape


(1307, 148)

In [13]:
# drop columns that have no predictive value 

t4 = t4.drop(columns=['patdeid'], axis=1)

In [14]:
t4[:5]

,medication,avg_weekly_dose,sru_alcohol_0_above_mean,sru_alcohol_0_below_mean,sru_alcohol_0_mean,sru_alcohol_0_none_reported,sru_alcohol_4_above_mean,sru_alcohol_4_below_mean,sru_alcohol_4_mean,sru_alcohol_4_none_reported,sru_cannabis_0_above_mean,sru_cannabis_0_below_mean,sru_cannabis_0_mean,sru_cannabis_0_none_reported,sru_cannabis_4_above_mean,sru_cannabis_4_below_mean,sru_cannabis_4_mean,sru_cannabis_4_none_reported,sru_cocaine_0_above_mean,sru_cocaine_0_below_mean,sru_cocaine_0_mean,sru_cocaine_0_none_reported,sru_cocaine_4_above_mean,sru_cocaine_4_below_mean,sru_cocaine_4_mean,sru_cocaine_4_none_reported,sru_amphetamine_0_above_mean,sru_amphetamine_0_below_mean,sru_amphetamine_0_none_reported,sru_amphetamine_4_above_mean,sru_amphetamine_4_below_mean,sru_amphetamine_4_mean,sru_amphetamine_4_none_reported,sru_methamphetamine_0_above_mean,sru_methamphetamine_0_below_mean,sru_methamphetamine_0_none_reported,sru_methamphetamine_4_above_mean,sru_methamphetamine_4_below_mean,sru_methamphetamine_4_mean,sru_methamphetamine_4_none_reported,sru_opiates_0_above_mean,sru_opiates_0_below_mean,sru_opiates_0_mean,sru_opiates_0_none_reported,sru_opiates_4_above_mean,sru_opiates_4_below_mean,sru_opiates_4_mean,sru_opiates_4_none_reported,sru_benzodiazepines_0_above_mean,sru_benzodiazepines_0_below_mean,sru_benzodiazepines_0_mean,sru_benzodiazepines_0_none_reported,sru_benzodiazepines_4_above_mean,sru_benzodiazepines_4_below_mean,sru_benzodiazepines_4_none_reported,sru_methadone_0_above_mean,sru_methadone_0_below_mean,sru_methadone_0_mean,sru_methadone_0_none_reported,sru_methadone_4_above_mean,sru_methadone_4_below_mean,sru_methadone_4_none_reported,sru_oxycodone_0_above_mean,sru_oxycodone_0_below_mean,sru_oxycodone_0_mean,sru_oxycodone_0_none_reported,sru_oxycodone_4_above_mean,sru_oxycodone_4_below_mean,sru_oxycodone_4_mean,sru_oxycodone_4_none_reported,sru_other_0_above_mean,sru_other_0_below_mean,sru_other_0_none_reported,sru_other_4_above_mean,sru_other_4_below_mean,sru_other_4_none_reported,t_Amphetamines_0_0.0,t_Amphetamines_0_1.0,t_Amphetamines_1_0.0,t_Amphetamines_1_1.0,t_Amphetamines_2_0.0,t_Amphetamines_2_1.0,t_Amphetamines_3_0.0,t_Amphetamines_3_1.0,t_Amphetamines_4_0.0,t_Amphetamines_4_1.0,t_Benzodiazapines_0_0.0,t_Benzodiazapines_0_1.0,t_Benzodiazapines_1_0.0,t_Benzodiazapines_1_1.0,t_Benzodiazapines_2_0.0,t_Benzodiazapines_2_1.0,t_Benzodiazapines_3_0.0,t_Benzodiazapines_3_1.0,t_Benzodiazapines_4_0.0,t_Benzodiazapines_4_1.0,t_Oxycodone_0_0.0,t_Oxycodone_0_1.0,t_Oxycodone_1_0.0,t_Oxycodone_1_1.0,t_Oxycodone_2_0.0,t_Oxycodone_2_1.0,t_Oxycodone_3_0.0,t_Oxycodone_3_1.0,t_Oxycodone_4_0.0,t_Oxycodone_4_1.0,t_Cocaine_0_0.0,t_Cocaine_0_1.0,t_Cocaine_1_0.0,t_Cocaine_1_1.0,t_Cocaine_2_0.0,t_Cocaine_2_1.0,t_Cocaine_3_0.0,t_Cocaine_3_1.0,t_Cocaine_4_0.0,t_Cocaine_4_1.0,t_Methamphetamine_0_0.0,t_Methamphetamine_0_1.0,t_Methamphetamine_1_0.0,t_Methamphetamine_1_1.0,t_Methamphetamine_2_0.0,t_Methamphetamine_2_1.0,t_Methamphetamine_3_0.0,t_Methamphetamine_3_1.0,t_Methamphetamine_4_0.0,t_Methamphetamine_4_1.0,t_Opiate300_0_0.0,t_Opiate300_0_1.0,t_Opiate300_1_0.0,t_Opiate300_1_1.0,t_Opiate300_2_0.0,t_Opiate300_2_1.0,t_Opiate300_3_0.0,t_Opiate300_3_1.0,t_Opiate300_4_0.0,t_Opiate300_4_1.0,t_Cannabinoids_0_0.0,t_Cannabinoids_0_1.0,t_Cannabinoids_1_0.0,t_Cannabinoids_1_1.0,t_Cannabinoids_2_0.0,t_Cannabinoids_2_1.0,t_Cannabinoids_3_0.0,t_Cannabinoids_3_1.0,t_Cannabinoids_4_0.0,t_Cannabinoids_4_1.0,outcome
0,2.0,208.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0
1,2.0,78.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [15]:
# check value counts for dtypes, make sure there are no strings
t4.dtypes.value_counts()

int64      144
float64      3
Name: count, dtype: int64

### Data is cleaned properly and ready for machine learning

In [16]:
# check shape and head 
display(t4.shape)
display(t4.head())


(1307, 147)

,medication,avg_weekly_dose,sru_alcohol_0_above_mean,sru_alcohol_0_below_mean,sru_alcohol_0_mean,sru_alcohol_0_none_reported,sru_alcohol_4_above_mean,sru_alcohol_4_below_mean,sru_alcohol_4_mean,sru_alcohol_4_none_reported,sru_cannabis_0_above_mean,sru_cannabis_0_below_mean,sru_cannabis_0_mean,sru_cannabis_0_none_reported,sru_cannabis_4_above_mean,sru_cannabis_4_below_mean,sru_cannabis_4_mean,sru_cannabis_4_none_reported,sru_cocaine_0_above_mean,sru_cocaine_0_below_mean,sru_cocaine_0_mean,sru_cocaine_0_none_reported,sru_cocaine_4_above_mean,sru_cocaine_4_below_mean,sru_cocaine_4_mean,sru_cocaine_4_none_reported,sru_amphetamine_0_above_mean,sru_amphetamine_0_below_mean,sru_amphetamine_0_none_reported,sru_amphetamine_4_above_mean,sru_amphetamine_4_below_mean,sru_amphetamine_4_mean,sru_amphetamine_4_none_reported,sru_methamphetamine_0_above_mean,sru_methamphetamine_0_below_mean,sru_methamphetamine_0_none_reported,sru_methamphetamine_4_above_mean,sru_methamphetamine_4_below_mean,sru_methamphetamine_4_mean,sru_methamphetamine_4_none_reported,sru_opiates_0_above_mean,sru_opiates_0_below_mean,sru_opiates_0_mean,sru_opiates_0_none_reported,sru_opiates_4_above_mean,sru_opiates_4_below_mean,sru_opiates_4_mean,sru_opiates_4_none_reported,sru_benzodiazepines_0_above_mean,sru_benzodiazepines_0_below_mean,sru_benzodiazepines_0_mean,sru_benzodiazepines_0_none_reported,sru_benzodiazepines_4_above_mean,sru_benzodiazepines_4_below_mean,sru_benzodiazepines_4_none_reported,sru_methadone_0_above_mean,sru_methadone_0_below_mean,sru_methadone_0_mean,sru_methadone_0_none_reported,sru_methadone_4_above_mean,sru_methadone_4_below_mean,sru_methadone_4_none_reported,sru_oxycodone_0_above_mean,sru_oxycodone_0_below_mean,sru_oxycodone_0_mean,sru_oxycodone_0_none_reported,sru_oxycodone_4_above_mean,sru_oxycodone_4_below_mean,sru_oxycodone_4_mean,sru_oxycodone_4_none_reported,sru_other_0_above_mean,sru_other_0_below_mean,sru_other_0_none_reported,sru_other_4_above_mean,sru_other_4_below_mean,sru_other_4_none_reported,t_Amphetamines_0_0.0,t_Amphetamines_0_1.0,t_Amphetamines_1_0.0,t_Amphetamines_1_1.0,t_Amphetamines_2_0.0,t_Amphetamines_2_1.0,t_Amphetamines_3_0.0,t_Amphetamines_3_1.0,t_Amphetamines_4_0.0,t_Amphetamines_4_1.0,t_Benzodiazapines_0_0.0,t_Benzodiazapines_0_1.0,t_Benzodiazapines_1_0.0,t_Benzodiazapines_1_1.0,t_Benzodiazapines_2_0.0,t_Benzodiazapines_2_1.0,t_Benzodiazapines_3_0.0,t_Benzodiazapines_3_1.0,t_Benzodiazapines_4_0.0,t_Benzodiazapines_4_1.0,t_Oxycodone_0_0.0,t_Oxycodone_0_1.0,t_Oxycodone_1_0.0,t_Oxycodone_1_1.0,t_Oxycodone_2_0.0,t_Oxycodone_2_1.0,t_Oxycodone_3_0.0,t_Oxycodone_3_1.0,t_Oxycodone_4_0.0,t_Oxycodone_4_1.0,t_Cocaine_0_0.0,t_Cocaine_0_1.0,t_Cocaine_1_0.0,t_Cocaine_1_1.0,t_Cocaine_2_0.0,t_Cocaine_2_1.0,t_Cocaine_3_0.0,t_Cocaine_3_1.0,t_Cocaine_4_0.0,t_Cocaine_4_1.0,t_Methamphetamine_0_0.0,t_Methamphetamine_0_1.0,t_Methamphetamine_1_0.0,t_Methamphetamine_1_1.0,t_Methamphetamine_2_0.0,t_Methamphetamine_2_1.0,t_Methamphetamine_3_0.0,t_Methamphetamine_3_1.0,t_Methamphetamine_4_0.0,t_Methamphetamine_4_1.0,t_Opiate300_0_0.0,t_Opiate300_0_1.0,t_Opiate300_1_0.0,t_Opiate300_1_1.0,t_Opiate300_2_0.0,t_Opiate300_2_1.0,t_Opiate300_3_0.0,t_Opiate300_3_1.0,t_Opiate300_4_0.0,t_Opiate300_4_1.0,t_Cannabinoids_0_0.0,t_Cannabinoids_0_1.0,t_Cannabinoids_1_0.0,t_Cannabinoids_1_1.0,t_Cannabinoids_2_0.0,t_Cannabinoids_2_1.0,t_Cannabinoids_3_0.0,t_Cannabinoids_3_1.0,t_Cannabinoids_4_0.0,t_Cannabinoids_4_1.0,outcome
0,2.0,208.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0
1,2.0,78.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [ ]:
# data is ready for machine learning

t4.to_csv('/Users/danherman/Desktop/oud_treatment_outcome/data/clean_data/final167.csv', index=False)